In [ ]:
!pip install textblob
!pip install nltk
!pip install gensim

In [ ]:
!python -m textblob.download_corpora lite

In [18]:
import pandas as pd
from textblob import TextBlob
import nltk
import gensim
import numpy as np
import re

In [ ]:
df = pd.read_csv('EarningsDataSentences Dojo.csv')

In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
stop_en = stopwords.words('english')

In [6]:
df.head(10)

,ID,Sentence,Keyword,Health,Climate,Inequality,Pollution,Waste,Poverty,Classification,cleanedsentence
0,2028351,We believe we could offer greater performance ...,batteries,0,0,0,1,0,0,0,We believe we could offer greater performance ...
1,2028351,""", 'Meanwhile, many other companies continue t...",batteries,0,0,0,1,0,0,0,Meanwhile many other companies continue to te...
2,2028351,Our batteries are a drop-in replacement and re...,batteries,0,0,0,1,0,0,0,Our batteries are a dropin replacement and ret...
3,2028351,"', 'We are also continuing to work with our OE...",batteries,0,0,0,1,0,0,1,We are also continuing to work with our OEM p...
4,2028351,We're managing our working capital very carefu...,health,1,0,0,0,0,0,0,Were managing our working capital very careful...
5,2028351,"', 'We began an intensive product development ...",batteries,0,0,0,1,0,0,1,We began an intensive product development per...
6,2028351,Walmart spent five months testing our batterie...,batteries,0,0,0,1,0,0,0,Walmart spent five months testing our batterie...
7,2028351,Then the company had the materials handling ve...,batteries,0,0,0,1,0,0,0,Then the company had the materials handling ve...
8,2028351,"Typically, they test our batteries in their mo...",batteries,0,0,0,1,0,0,0,Typically they test our batteries in their mos...
9,2028351,"', ""They also ask for a confirmation from the ...",batteries,0,0,0,1,0,0,0,They also ask for a confirmation from the mat...


In [15]:
df_train = df.iloc[:15000,:]

In [16]:
def clean_text(text):
    t = TextBlob(text.lower()).tokenize()
    t = [w for w in t if re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', w) and w not in stop_en]
    t = t.lemmatize()
    return " ".join(t)

In [73]:
df_train['txt2'] = df['cleanedsentence'].apply(lambda x: clean_text(x))

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [74]:
list(df_train['txt_nouns'].tolist())

[WordList(['believe', 'could', 'offer', 'greater', 'performance', 'value', 'intensive', 'user', 'superior', 'present', 'power', 'source', 'like', 'lead', 'acid', 'batteries', 'hydrogen', 'propane', 'lithium', 'ion']),
 WordList(['meanwhile', 'many', 'companies', 'continue', 'test', 'batteries']),
 WordList(['batteries', 'dropin', 'replacement', 'retrofits', 'existing', 'electric', 'materialhandling', 'vehicle']),
 WordList(['also', 'continuing', 'work', 'oem', 'partners', 'develop', 'new', 'products', 'markets', 'using', 'batteries']),
 WordList(['managing', 'working', 'capital', 'carefully', 'maintain', 'balance', 'sheet', 'health', 'wait', 'oems', 'direct', 'battery', 'sales', 'accelerate']),
 WordList(['began', 'intensive', 'product', 'development', 'period', 'launching', 'initial', 'forklift', 'batteries', 'fiscal']),
 WordList(['walmart', 'spent', 'five', 'months', 'testing', 'batteries', 'room', 'temperature', 'also', 'cold', 'warehouses']),
 WordList(['company', 'materials', 'ha

In [75]:
#global_words = pd.Series(" ".join(df_train['txt2']).split()).value_counts()
global_words

growth                  1926
health                  1890
business                1563
year                    1537
healthy                 1528
healthcare              1442
also                    1298
quarter                 1235
million                 1235
community               1175
new                     1142
well                    1107
impact                  1051
market                  1001
cancer                   994
headwinds                881
continue                 833
raw                      793
think                    788
positive                 785
see                      754
strong                   707
revenue                  692
cost                     689
first                    685
material                 663
weve                     647
one                      645
headwind                 623
higher                   615
                        ... 
sixweek                    1
selfsustaining             1
aorta                      1
granule       

In [76]:
common_words = global_words[(global_words < 5) | (global_words > 500)]

In [77]:
def remove_common_words(text):
    t = TextBlob(text).tokenize()
    t = [w for w in t if w not in common_words]
    return t

In [78]:
df_train['txt2'] = df_train['txt2'].apply(lambda x: remove_common_words(x))

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [79]:
data_text = list(df_train['txt2'])

In [80]:
bigram = gensim.models.Phrases(data_text, min_count=5, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_text], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [81]:
data_text = [trigram_mod[bigram_mod[doc]] for doc in data_text]

In [82]:
data_text

[['believe',
  'could',
  'offer',
  'greater',
  'performance',
  'value',
  'intensive',
  'user',
  'superior',
  'present',
  'power',
  'source',
  'like',
  'lead',
  'batteries',
  'hydrogen',
  'lithium_ion'],
 ['meanwhile', 'many', 'companies', 'test', 'batteries'],
 ['batteries', 'replacement', 'existing', 'electric_vehicle'],
 ['continuing', 'work', 'oem', 'partners', 'develop', 'using', 'batteries'],
 ['managing',
  'working_capital',
  'carefully',
  'maintain',
  'balance_sheet',
  'wait',
  'oems',
  'direct',
  'battery',
  'accelerate'],
 ['began',
  'intensive',
  'product',
  'development',
  'period',
  'launching',
  'initial',
  'batteries',
  'fiscal'],
 ['spent', 'five', 'months', 'testing', 'batteries', 'room', 'cold'],
 ['company',
  'materials',
  'handling',
  'vehicle',
  'makers',
  'test',
  'batteries',
  'months'],
 ['typically',
  'test',
  'batteries',
  'intensive',
  'application',
  'takes',
  'months'],
 ['ask',
  'materials',
  'handling',
  'veh

# Sentence Clustering

In [83]:
from gensim import corpora
dictionary = corpora.Dictionary(data_text)
corpus = [dictionary.doc2bow(text) for text in data_text]

In [84]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word=dictionary, passes=15, random_state=100, alpha='auto')
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.013*"fourth" + 0.012*"portfolio" + 0.011*"lower" + 0.011*"significant"')
(1, '0.013*"time" + 0.013*"like" + 0.012*"thats" + 0.012*"really"')
(2, '0.018*"strategy" + 0.009*"grew" + 0.008*"pleased" + 0.008*"clients"')
(3, '0.012*"continued" + 0.012*"demand" + 0.011*"product" + 0.011*"performance"')
(4, '0.012*"project" + 0.012*"data" + 0.010*"safety" + 0.008*"water"')
(5, '0.013*"company" + 0.012*"solar" + 0.011*"services" + 0.011*"ecosystem"')


In [63]:
!pip install pyldavis

    100% |████████████████████████████████| 1.6MB 1.3MB/s ta 0:00:01    19% |██████                          | 307kB 1.3MB/s eta 0:00:02
  Running setup.py bdist_wheel for pyldavis ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


In [85]:
lda = gensim.models.ldamodel.LdaModel.load('model10.gensim')

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [102]:
results = []
for i in range(len(data_text)):
    results.append(ldamodel.get_document_topics(dictionary.doc2bow(data_text[i])))

In [103]:
pd.DataFrame(results)

,0,1,2,3,4,5
0,"(0, 0.069560744)","(1, 0.18759698)","(2, 0.03001651)","(3, 0.20248279)","(4, 0.13958567)","(5, 0.37075728)"
1,"(0, 0.13103054)","(1, 0.19976789)","(2, 0.05730034)","(3, 0.21182358)","(4, 0.2140945)","(5, 0.18598312)"
2,"(0, 0.1418266)","(1, 0.20492123)","(2, 0.06237853)","(3, 0.19751228)","(4, 0.23244137)","(5, 0.16092001)"
3,"(0, 0.11480839)","(1, 0.109134845)","(2, 0.04972171)","(3, 0.1322627)","(4, 0.14448589)","(5, 0.44958645)"
4,"(0, 0.30308583)","(1, 0.19179413)","(2, 0.09668657)","(3, 0.105039746)","(4, 0.15806942)","(5, 0.14532432)"
5,"(0, 0.2755194)","(1, 0.09049196)","(2, 0.04395957)","(3, 0.17850748)","(4, 0.25445753)","(5, 0.15706411)"
6,"(0, 0.15942207)","(1, 0.20565397)","(2, 0.04972447)","(3, 0.11192839)","(4, 0.34387738)","(5, 0.12939374)"
7,"(0, 0.11440025)","(1, 0.124332584)","(2, 0.046644885)","(3, 0.22598663)","(4, 0.32583338)","(5, 0.16280228)"
8,"(0, 0.15752369)","(1, 0.13134861)","(2, 0.04972445)","(3, 0.11170283)","(4, 0.43620488)","(5, 0.11349557)"
9,"(0, 0.078979775)","(1, 0.15681654)","(2, 0.034362208)","(3, 0.4866437)","(4, 0.1672558)","(5, 0.075941995)"


In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [107]:
vectorizer = TfidfVectorizer()

In [112]:
vectorizer.fit(df_train['txt2'].apply(lambda x: " ".join(x)))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [118]:
vectorizer.

{'believe': 426,
 'could': 986,
 'offer': 2913,
 'greater': 1928,
 'performance': 3130,
 'value': 4644,
 'intensive': 2268,
 'user': 4620,
 'superior': 4277,
 'present': 3300,
 'power': 3267,
 'source': 4090,
 'like': 2466,
 'lead': 2411,
 'batteries': 404,
 'hydrogen': 2089,
 'lithium': 2487,
 'ion': 2309,
 'meanwhile': 2618,
 'many': 2571,
 'companies': 815,
 'test': 4369,
 'replacement': 3672,
 'existing': 1573,
 'electric': 1380,
 'vehicle': 4658,
 'continuing': 941,
 'work': 4793,
 'oem': 2911,
 'partners': 3081,
 'develop': 1170,
 'using': 4623,
 'managing': 2564,
 'working': 4799,
 'capital': 582,
 'carefully': 597,
 'maintain': 2547,
 'balance': 387,
 'sheet': 3971,
 'wait': 4702,
 'oems': 2912,
 'direct': 1218,
 'battery': 405,
 'accelerate': 12,
 'began': 415,
 'product': 3360,
 'development': 1175,
 'period': 3135,
 'launching': 2407,
 'initial': 2206,
 'fiscal': 1728,
 'spent': 4126,
 'five': 1731,
 'months': 2756,
 'testing': 4371,
 'room': 3789,
 'cold': 755,
 'company': 